## LangChain 사용해서 지원사업 공고 파일(pdf, hwp, png)을 요약해보자!

In [83]:
from openai import OpenAI
from getpass import getpass
from tqdm import tqdm

from utils.get_text import get_hwp_text, get_pdf_text # pdf, hwp에서 텍스트를 추출하는 사용자 지정 함수 

- LangChain 모델이 참고할 비슷한 용어 사전
- 개발팀이 직접 정의
- (추후에 유사한 단어가 나오면 사전에 추가하는 기능도 구현할 예정)

In [85]:
# 정의한 비슷한 용어 사전
keyword_dictionary = {  
                        '사업자 유형' : ['개인사업자', '법인사업자', '창업(예비사업자 포함)', '재창업', '기존사업자'],
                        '지원업종' : ['자동차 및 부품 판매업','도매 및 상품 중개업','소매업(자동차 제외)','숙박업', '음식점업', '제조업','교육 서비스업', '협회 및 단체, 수리 및 기타 개인 서비스업','부동산업','전문, 과학 및 기술 서비스업','예술, 스포츠 및 여가관련 서비스업','정보통신업','농업, 임업 및 어업','건설업','운수 및 창고업','보건업 및 사회복지 서비스업','사업시설 관리, 사업 지원 및 임대 서비스업','금융 및 보험업','전기, 가스, 증기 및 공기 조절 공급업','광업','수도, 하수 및 폐기물 처리, 원료 재생업','가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동','공공행정, 국방 및 사회보장 행정','국제 및 외국기관'],
                        '정보통신업' : ['정보통신 서비스업', '정보통신기기 제조업', '소프트웨어 및 컴퓨터 관련 서비스업'],
                        '공고 특성' : ['청년 대상', '여성 대상', '소상공인', '대출', '마케팅·홍보', '보조금', '폐업', '고용지원', '시설·환경 개선', '입주·임대 지원', '희망리턴패키지', '고용유지지원금', '희망대출플러스', '두루누리지원금', '창업패키지', '노란우산공제', '이커머스 입점피해'],
                        '창업패키지' : ['예비창업패키지', '초기창업패키지', '청년창업사관학교'],
                        '신청기간' : ['신청 기간', '접수 기간', '자금 소진시까지'],
                        '신청방법' : ['지원 대상', '사업 대상', '융자 대상', '접수방법','접수'],
                        '지원제외' : ['지원 제외','지원 취소', '제외 대상', '제한 대상'],
                        '우대사항' : ['벤처기업', '여성기업', '사회적기업(인증)', '장애인기업', '이노비즈제도','메인비즈제도','가족친화인증기업', '수출유망중소기업','녹색기술인증기업','내일채움공제 가입기업','글로벌 강소기업', '기술혁신중소기업'],
                        '유의사항' : ['의무사항', '지원 취소 및 제외', '기타 안내사항'],
                        '제출 서류' : ['필요 서류', '서류 제출 목록'],
                        '선정 절차' : ['선정 절차', '평가 방법', '평가 방향', '우대 사항', '평가 방향', '선정 기준', '지원 절차'],
                        '지원 내용' : ['지원 내용', '규모', '분야 및 대상', '범위 및 형태', '세부지원내용', '지원조건'],
                        '문의처' : ['문의 전화', '연락처', '지원 문의처', '문의 및 상담', '접수 및 문의', '접수'],
                        '기타' : ['기타']
                        }

#### model 구현 준비
- OpenAI 객체 생성
- 파일 텍스트 추출
- model이 참고할 기본 템플릿 작성

In [86]:
# 개인의 API key를 입력
MY_API_KEY = getpass("OpenAI API Key")

#OpenAI 객체 생성
client = OpenAI(api_key=MY_API_KEY)

In [87]:
# 파일 경로 설정 (백슬래시 문제 해결을 위해 r"" 사용)
file_path = f"금융/[경기] 부천시 2025년 소상공인 특례보증ㆍ이차보전 지원계획 공고.pdf"

# 지원사업명 저장 (지원경로를 분할하여 사용)
title = file_path.split('/')[1].split('.')[0]

# 파일 타입 저장 (지원경로를 분할하여 사용)
file_type = file_path.split('/')[1].split('.')[1]

# file_type이 pdf면 get_pdf_text, hwp면 get_hwp_text를 사용하여 텍스트 추출
if file_type == 'pdf':
    full_text = get_pdf_text(file_path)
elif file_type == 'hwp':
    full_text = get_hwp_text(file_path)

In [88]:
base_template = {f'''
📌 **{title} 요약 안내**

✅ **사업자 유형**

☑️ **지원업종**

💡 **공고특성**

👥 **지원대상**

📅 **신청기간**  

📝 **신청방법**  

🚫 **지원제외 대상**

⭐️ **우대사항**

👩 **여성 대상**

💁 **청년 대상**

⚠ **유의사항**  

📑 **제출서류**  

📌 **선정절차**  

💰 **지원내용**  

📞 **문의처**

'''
}

# 📢 **기타 안내**  

In [89]:
explanation = {f'''
- 중소기업은 중소기업기본법에 정의된 기준을 따르며 개인사업자, 법인사업자, 기존사업자를 포함해주세요.
- 시설·환경 개선 자금은 기업 운영에 필요한 기계설비, 공장설립, 토지구입 등에 사용되는 자금, 원자재 구매, 생산, 판매 등 기업의 운영에 필요한 자금입니다.
- 청년은 만 19세 이상 39세 이하를 의미합니다. 
- 전용 대상은 특정 대상만 신청이 가능한 경우를 의미하고, 우대 조건은 특정 대상에게 가점을 부여하는 경우를 의미합니다.
- 공고에서 언급된 모든 금리 우대 항목을 추출해주세요. 
- 여성대상과 청년대상은 대상이 여성만 또는 청년만 되는 내용이 있는 경우에 해당하며 O,X로 구분해주세요.
- 만약 '변동없음'이라고 기재되어 있다면, 기존에 제공된 값을 찾아서 사용하세요. 즉, 당초(1,3,5,6,9월)이라면 기존 값을 그대로 추출하세요.
'''}

#### LangChain 모델로 간단하게 요약해보기
- (고도화 예정)

In [90]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
당신은 정부 지원 사업 전문 분석가입니다. 

사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 추출해야합니다. 파일 내용을 철저하게 분석하고, 어떤 항목도 누락시키지 마세요. 파일에 없는 내용은 공백으로 남겨두십시오.
요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니 사전에 해당하는 단어가 나오면 참고해주십시오.

사업자 유형, 지원업종, 공고특성, 우대사항 항목은 문맥을 분석해 가장 적합한 단어를 단어사전 내에서 선택해주세요.조건에 해당하는 사항이 중복되는 경우, 각 대상에 모두 포함시켜주세요.
다른 항목의 경우는 사전에 해당하지 않는 단어가 나오면, 해당 단어를 그대로 사용하여 요약해주십시오.

신청기간과 제출서류는 각 항목을 세분화할 수 있는 경우, 세분화하여 요약해주세요.

지원대상, 신청방법은 모든 내용을 누락없이 추출해주세요. 접수처와 접수처 현황까지 모두 포함해야 합니다.
신청기간과 제출서류가 자금마다 두 가지 이상으로 다르게 제시된 경우에만 '공고문 참조'로 대체해주세요. 만약 신청기간이나 제출서류가 하나로 통일되어 있거나, 하나의 기간 혹은 제출서류만 존재하는 경우에는 구체적인 값을 출력해주세요.

신청방법, 지원내용 항목의 경우 표로 제시된 정보는 그대로 표 형태로 출력해주세요. 표는 데이터프레임 형식으로 출력해주세요.

제공하는 템플릿 양식 : {base_template}
단어 정의 사전 : {keyword_dictionary}
참고 : {explanation}
---------------------------------------------------
사용자 요구 파일 : {text}

각 항목의 범위를 명확히 구분하고, 잘못 분류된 정보를 수정해주세요.
"""

# 파일 요약을 위한 프롬프트 템플릿을 정의
prompt = PromptTemplate.from_template(template)

# OpenAI의 GPT-4o 모델을 사용하여 채팅 모델을 생성
model = ChatOpenAI(
    model='gpt-4o',  # 사용할 언어 모델 지정
    api_key=MY_API_KEY,  # OpenAI API 키 입력
    temperature=0
)

# 모델의 출력을 문자열로 변환하는 출력 파서
output_parser = StrOutputParser()

# 체인(Chain) 구성: 프롬프트 → 모델 → 출력 파서 순서로 연결
chain = prompt | model | output_parser

In [91]:
print(chain.invoke({'base_template':base_template,
              'keyword_dictionary':keyword_dictionary,
                'explanation': explanation,
              'text':full_text}))

```
📌 **[경기] 부천시 2025년 소상공인 특례보증ㆍ이차보전 지원계획 공고 요약 안내**

✅ **사업자 유형**
- 소상공인
- 기존사업자

☑️ **지원업종**
- 소상공인

💡 **공고특성**
- 소상공인
- 대출

👥 **지원대상**
- 부천시 소재 소상공인
- 사업자 등록일로부터 2개월 이상 경과된 사업자

📅 **신청기간**
- 상반기: 2025. 1. 23. ~ 자금소진시까지
- 하반기: 2025. 7. 1. ~ 자금소진시까지

📝 **신청방법**
| 접수처 | 연락처 | 주소 |
|--------|--------|------|
| 경기신용보증재단 부천지점 | 1577-5900 | 부천시 원미구 소향로 217, 4층 |

🚫 **지원제외 대상**
- 지방세, 세외수입 등 체납이 있는 사업자
- 제한업종(금융업, 유흥업, 불법도박 및 사치, 향락업종 등)

⭐️ **우대사항**
- 

👩 **여성 대상**
- X

💁 **청년 대상**
- X

⚠ **유의사항**
- 사업장 휴․폐업, 사업장 타 시․군 이전 시 지원 중지 및 환수

📑 **제출서류**
- 사업자등록증 사본
- 매출증빙서류
- 신분증 등

📌 **선정절차**
- 상담․접수 ➡ 심사(서류․현장) ➡ 보증서발급 ➡ 자금대출
- 경기신용보증재단 ➡ 경기신용보증재단 ➡ 경기신용보증재단 ➡ 은행

💰 **지원내용**
- 특례보증: 사업자별 최대 5,000만 원
- 이차보전: 대출이자 연 2% 1년간 지원
- 협약은행: 농협은행, 하나은행, 국민은행, 기업은행, 신한은행, 우리은행

📞 **문의처**
- 경기신용보증재단 부천지점: 1577-5900
- 부천시 지역경제과: 032-625-2766
```
